In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [2]:
# a function to draw a plot of an SVM
def plot_svc(svc, X, y, h=0.02, pad=0.25):
    x_min, x_max = X[:, 0].min()-pad, X[:, 0].max()+pad
    y_min, y_max = X[:, 1].min()-pad, X[:, 1].max()+pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8, 5))
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)

    plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=mpl.cm.Paired)
    sv = svc.support_vectors_
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.show()

landmarks = pd.read_csv('./data/tidyLandmarks_no_na.csv')

In [3]:
def svm_classification(landmarks, index):
    # filter out the landmarks needed
    chosenLandmark = landmarks[landmarks.landmark_index==index]
    chosenLandmark = chosenLandmark[np.isfinite(chosenLandmark['r'])]
    
    # create training and testing data
    X = chosenLandmark[['pts', 'r']]
    y = chosenLandmark['stype']
    y = y.replace(['mt-zrf'], 1)
    y = y.replace(['wt-zrf'], 0)

    # check whether both classes exist
    count_1 = chosenLandmark['stype'].str.contains('mt-zrf').sum()
    count_0 = chosenLandmark['stype'].str.contains('wt-zrf').sum()

    print(count_1, count_0)
    if (count_1 < 2 or count_0 < 2):
        return None, None

    # present the data
    '''plt.figure(figsize=(8, 5))
    plt.scatter(X.values[:,0], X.values[:,1], s=70, c=y, cmap=mpl.cm.Paired)
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.show()'''
    
    # find the best C value by cross-validation
    tuned_parameters = [{'C': [0.01, 0.1, 1, 10, 100, 1000]}]
    clf = GridSearchCV(SVC(kernel='linear'), tuned_parameters, cv=10, scoring='accuracy')
    clf.fit(X.values, y.values)
    best_c = clf.best_params_['C']
    
    svc = SVC(C=best_c, kernel='linear')
    svc.fit(X, y)
    
    #plot_svc(svc, X.values, y)
    
    prediction = svc.predict(X)
    # print confusion matrix
    print("confusion matrix: ")
    cm = confusion_matrix(y, prediction)
    cm_df = pd.DataFrame(cm.T, index=svc.classes_, columns=svc.classes_)
    print(cm_df)
    print('Classification Report: ')
    print(classification_report(y, 
                            prediction,
                            digits = 3))
    # print accuracy score
    accuracy = f1_score(y, prediction)
    
    return svc, accuracy

In [5]:
sample_1 = landmarks[landmarks.sample_index==101]
sample_1 = sample_1[np.isfinite(sample_1['r'])]
landmarks_1 = sample_1['landmark_index']

results = []
for l in landmarks_1.values:
    print ("=======================================")
    print ("landmark: ", str(l))
    svc, accuracy = svm_classification(landmarks[landmarks.sample_index!=1], l)
    if (svc is None or accuracy is None):
        print("One of the classes have too few samples for this landmark, so skipping it.")
        continue
    prediction = svc.predict(sample_1[sample_1.landmark_index==1][['pts', 'r']])
    results.append((l, prediction[0], accuracy))
    print(results)

print ("=======================================")
print("SAMPLE REPORT")
r_sig = []
for svm in results:
    if svm[2]>0.5:
        r_sig.append(svm)

print("The sample has", str(len(results)), " landmarks.")
print(str(len(r_sig)), " landmarks have f1 score larger than 0.5.")

one = []
zero = []
for svm in r_sig:
    if svm[1]==1:
        one.append(svm)
    else:
        zero.append(svm)

print("One was voted ", str(len(one)), " times.")
print("Zero was voted ", str(len(zero)), " times.")

landmark:  1
10 46
confusion matrix: 
    0  1
0  45  6
1   1  4
Classification Report: 
             precision    recall  f1-score   support

          0      0.882     0.978     0.928        46
          1      0.800     0.400     0.533        10

avg / total      0.868     0.875     0.857        56

[(1, 0, 0.5333333333333333)]
landmark:  2
14 42
confusion matrix: 
    0   1
0  41   4
1   1  10
Classification Report: 
             precision    recall  f1-score   support

          0      0.911     0.976     0.943        42
          1      0.909     0.714     0.800        14

avg / total      0.911     0.911     0.907        56

[(1, 0, 0.5333333333333333), (2, 1, 0.8)]
landmark:  3
10 46
confusion matrix: 
    0  1
0  45  7
1   1  3
Classification Report: 
             precision    recall  f1-score   support

          0      0.865     0.978     0.918        46
          1      0.750     0.300     0.429        10

avg / total      0.845     0.857     0.831        56

[(1, 0, 0.5333

/Users/jyun38/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jyun38/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


confusion matrix: 
    0   1
0  39   3
1   3  11
Classification Report: 
             precision    recall  f1-score   support

          0      0.929     0.929     0.929        42
          1      0.786     0.786     0.786        14

avg / total      0.893     0.893     0.893        56

[(1, 0, 0.5333333333333333), (2, 1, 0.8), (3, 0, 0.4285714285714285), (4, 0, 0.8461538461538461), (5, 0, 0.47058823529411764), (6, 0, 0.6363636363636364), (7, 0, 0.4), (8, 0, 0.8275862068965518), (9, 0, 0.6666666666666665), (10, 0, 0.5714285714285714), (11, 0, 0.625), (12, 0, 0.8571428571428571), (13, 0, 0.0), (14, 0, 0.7857142857142857)]
landmark:  15
10 46
confusion matrix: 
    0  1
0  43  5
1   3  5
Classification Report: 
             precision    recall  f1-score   support

          0      0.896     0.935     0.915        46
          1      0.625     0.500     0.556        10

avg / total      0.847     0.857     0.851        56

[(1, 0, 0.5333333333333333), (2, 1, 0.8), (3, 0, 0.428571428571428

confusion matrix: 
    0   1
0  46  10
1   0   0
Classification Report: 
             precision    recall  f1-score   support

          0      0.821     1.000     0.902        46
          1      0.000     0.000     0.000        10

avg / total      0.675     0.821     0.741        56

[(1, 0, 0.5333333333333333), (2, 1, 0.8), (3, 0, 0.4285714285714285), (4, 0, 0.8461538461538461), (5, 0, 0.47058823529411764), (6, 0, 0.6363636363636364), (7, 0, 0.4), (8, 0, 0.8275862068965518), (9, 0, 0.6666666666666665), (10, 0, 0.5714285714285714), (11, 0, 0.625), (12, 0, 0.8571428571428571), (13, 0, 0.0), (14, 0, 0.7857142857142857), (15, 0, 0.5555555555555556), (16, 0, 0.7741935483870968), (17, 0, 0.4285714285714285), (18, 0, 0.6428571428571429), (19, 0, 0.6666666666666665), (20, 0, 0.7692307692307692), (21, 0, 0.5333333333333333), (22, 0, 0.5185185185185186), (23, 0, 0.33333333333333337), (24, 0, 0.6086956521739131), (25, 0, 0.0)]
landmark:  26
14 42


/Users/jyun38/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jyun38/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


...

confusion matrix: 
    0  1
0  41  1
1   5  9
Classification Report: 
             precision    recall  f1-score   support

          0      0.976     0.891     0.932        46
          1      0.643     0.900     0.750        10

avg / total      0.917     0.893     0.899        56

[(1, 0, 0.5333333333333333), (2, 1, 0.8), (3, 0, 0.4285714285714285), (4, 0, 0.8461538461538461), (5, 0, 0.47058823529411764), (6, 0, 0.6363636363636364), (7, 0, 0.4), (8, 0, 0.8275862068965518), (9, 0, 0.6666666666666665), (10, 0, 0.5714285714285714), (11, 0, 0.625), (12, 0, 0.8571428571428571), (13, 0, 0.0), (14, 0, 0.7857142857142857), (15, 0, 0.5555555555555556), (16, 0, 0.7741935483870968), (17, 0, 0.4285714285714285), (18, 0, 0.6428571428571429), (19, 0, 0.6666666666666665), (20, 0, 0.7692307692307692), (21, 0, 0.5333333333333333), (22, 0, 0.5185185185185186), (23, 0, 0.33333333333333337), (24, 0, 0.6086956521739131), (25, 0, 0.0), (26, 0, 0.5), (27, 0, 0.5333333333333333), (28, 0, 0.7692307692307692

confusion matrix: 
    0   1
0  42  14
1   0   0
Classification Report: 
             precision    recall  f1-score   support

          0      0.750     1.000     0.857        42
          1      0.000     0.000     0.000        14

avg / total      0.562     0.750     0.643        56

[(1, 0, 0.5333333333333333), (2, 1, 0.8), (3, 0, 0.4285714285714285), (4, 0, 0.8461538461538461), (5, 0, 0.47058823529411764), (6, 0, 0.6363636363636364), (7, 0, 0.4), (8, 0, 0.8275862068965518), (9, 0, 0.6666666666666665), (10, 0, 0.5714285714285714), (11, 0, 0.625), (12, 0, 0.8571428571428571), (13, 0, 0.0), (14, 0, 0.7857142857142857), (15, 0, 0.5555555555555556), (16, 0, 0.7741935483870968), (17, 0, 0.4285714285714285), (18, 0, 0.6428571428571429), (19, 0, 0.6666666666666665), (20, 0, 0.7692307692307692), (21, 0, 0.5333333333333333), (22, 0, 0.5185185185185186), (23, 0, 0.33333333333333337), (24, 0, 0.6086956521739131), (25, 0, 0.0), (26, 0, 0.5), (27, 0, 0.5333333333333333), (28, 0, 0.7692307692307

/Users/jyun38/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/jyun38/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


confusion matrix: 
    0  1
0  44  7
1   2  3
Classification Report: 
             precision    recall  f1-score   support

          0      0.863     0.957     0.907        46
          1      0.600     0.300     0.400        10

avg / total      0.816     0.839     0.817        56

[(1, 0, 0.5333333333333333), (2, 1, 0.8), (3, 0, 0.4285714285714285), (4, 0, 0.8461538461538461), (5, 0, 0.47058823529411764), (6, 0, 0.6363636363636364), (7, 0, 0.4), (8, 0, 0.8275862068965518), (9, 0, 0.6666666666666665), (10, 0, 0.5714285714285714), (11, 0, 0.625), (12, 0, 0.8571428571428571), (13, 0, 0.0), (14, 0, 0.7857142857142857), (15, 0, 0.5555555555555556), (16, 0, 0.7741935483870968), (17, 0, 0.4285714285714285), (18, 0, 0.6428571428571429), (19, 0, 0.6666666666666665), (20, 0, 0.7692307692307692), (21, 0, 0.5333333333333333), (22, 0, 0.5185185185185186), (23, 0, 0.33333333333333337), (24, 0, 0.6086956521739131), (25, 0, 0.0), (26, 0, 0.5), (27, 0, 0.5333333333333333), (28, 0, 0.7692307692307692

KeyboardInterrupt: 